In [ ]:
import requests
from time import sleep
from bs4 import BeautifulSoup

from tqdm import tqdm

import pandas as pd

In [ ]:
WEB_HEADERS = {'authority': 'auto.drom.ru', 'cache-control': 'no-cache',
               'upgrade-insecure-requests': '1',
               'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWeb'
                             'Kit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45'
                             'Safari/537.36',
               'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,'
                         'image/webp,image/apng,*/*;q=0.8,application/signed-'
                         'exchange;v=b3;q=0.9',
               'sec-fetch-site': 'none', 'sec-fetch-mode': 'navigate',
               'sec-fetch-user': '?1', 'sec-fetch-dest': 'document',
               'accept-language': 'en-US,en;q=0.9,ru-RU;q=0.8,ru;q=0.7'}

In [ ]:
def html_response(url, headers):
    for _ in range(3):
        try:
            response = requests.get(url, headers=headers)
            return response.text
        except ConnectionError or TimeoutError or ConnectionResetError:
            print("\n*****ConnectionError, TimeoutError or ConnectionResetError*"
                  "****\n\nI will retry again after 7 seconds...")
            sleep(5)
            print('Making another request...')


def parse_car_page_response(response):
    soup = BeautifulSoup(response, 'html.parser')
    selling_cars = []
    for full_header in soup.find_all('a', class_="css-1psewqh ewrty961", href=True):
        split_header = full_header['href']
        selling_cars.append(split_header)
    return selling_cars

In [ ]:
all_moscow_cars = [['https://moscow.drom.ru/audi/used/all/page', 34], ['https://moscow.drom.ru/bmw/used/all/page', 61],
                   ['https://moscow.drom.ru/chevrolet/used/all/page', 24], ['https://moscow.drom.ru/citroen/used/all/page', 10],
                   ['https://moscow.drom.ru/dodge/used/all/page', 6], ['https://moscow.drom.ru/ford/used/all/page', 47],
                   ['https://moscow.drom.ru/great_wall/used/all/page', 5], ['https://moscow.drom.ru/honda/used/all/page', 16],
                   ['https://moscow.drom.ru/hyundai/used/all/page', 85], ['https://moscow.drom.ru/infiniti/used/all/page', 9],
                   ['https://moscow.drom.ru/jeep/used/all/page', 6], ['https://moscow.drom.ru/kia/used/all/page', 100],
                   ['https://moscow.drom.ru/land_rover/used/all/page', 19], ['https://moscow.drom.ru/lexus/used/all/page', 15],
                   ['https://moscow.drom.ru/mazda/used/all/page', 22], ['https://moscow.drom.ru/mercedes-benz/used/all/page', 65],
                   ['https://moscow.drom.ru/mitsubishi/used/all/page', 34], ['https://moscow.drom.ru/nissan/used/all/page', 64],
                   ['https://moscow.drom.ru/opel/used/all/page', 24], ['https://moscow.drom.ru/peugeot/used/all/page', 11],
                   ['https://moscow.drom.ru/porsche/used/all/page', 10], ['https://moscow.drom.ru/renault/used/all/page', 38],
                   ['https://moscow.drom.ru/skoda/used/all/page', 43], ['https://moscow.drom.ru/ssang_yong/used/all/page', 7],
                   ['https://moscow.drom.ru/subaru/used/all/page', 6], ['https://moscow.drom.ru/suzuki/used/all/page', 11],
                   ['https://moscow.drom.ru/toyota/used/all/page', 59], ['https://moscow.drom.ru/volkswagen/used/all/page', 66],
                   ['https://moscow.drom.ru/volvo/used/all/page', 21], ['https://moscow.drom.ru/lada/used/all/page', 38],
                   ['https://moscow.drom.ru/uaz/used/all/page', 6]]

In [ ]:
links_to_lists_of_all_selling_cars = []

for car in all_moscow_cars:
  i = 1
  while i <= car[1]:
    links_to_lists_of_all_selling_cars.append(car[0]+str(i)+'/?unsold=1')
    i += 1

In [ ]:
car_page_response_list = []

for selling_cars_for_each_manufacturer in tqdm(links_to_lists_of_all_selling_cars):
  car_page_response = html_response(selling_cars_for_each_manufacturer, WEB_HEADERS)
  car_page_response_list.append(car_page_response)

100%|██████████| 962/962 [07:04<00:00,  2.27it/s]


In [ ]:
links_to_each_page_of_selling_car = []

for item in tqdm(car_page_response_list):
  links_to_each_page_of_selling_car.append(parse_car_page_response(item))

links_to_each_page_of_selling_car = [item for sublist in links_to_each_page_of_selling_car for item in sublist]

100%|██████████| 962/962 [01:38<00:00,  9.78it/s]


In [ ]:
car_data = []

for car_link in tqdm(links_to_each_page_of_selling_car):
  car_page = html_response(car_link, WEB_HEADERS)

  soup = BeautifulSoup(car_page, 'html.parser')

  try:
    brand = soup.body.h1.find_all_next(string=True)[0].split(', ')[0].replace("Продажа ", "").split(' ')[0]
  except:
    brand = "missing"

  try:
    model = soup.body.h1.find_all_next(string=True)[0].split(', ')[0].replace("Продажа ", "").split(' ')[1]
  except:
    model = "missing"

  try:
    manufacture_year = soup.body.h1.find_all_next(string=True)[0].split(', ')[1].split(' ')[0]
  except:
    manufacture_year = "missing"

  try:
    sellers_city = soup.body.h1.find_all_next(string=True)[0].split(', ')[1].split(' ')[-1]
  except:
    sellers_city = "missing"

  try:
    price = soup.find("div", class_="css-1003rx0 e162wx9x0").text.replace("\xa0", "").replace("₽", "")
  except:
    price = "missing"

  try:
    fuel_type = soup.find("span", class_="css-hgkucx e162wx9x0").text.split(", ")[0]
  except:
    fuel_type = "missing"

  try:
    engine_size = soup.find("span", class_="css-hgkucx e162wx9x0").text.split(", ")[1].replace(" л", "")
  except:
    engine_size = "missing"

  try:
    hp_power = soup.find("span", class_="css-z7plf3 e162wx9x0").text.split(",\xa0")[0].replace("\xa0л.с.", "")
  except:
    hp_power = "missing"

  try:
    gearbox_type = soup.find_all("td", class_="css-7whdrf ezjvm5n1")[2].text
  except:
    gearbox_type = "missing"

  try:
    transmission_type = soup.find_all("td", class_="css-7whdrf ezjvm5n1")[3].text
  except:
    transmission_type = "missing"

  try:
    color = soup.find_all("td", class_="css-7whdrf ezjvm5n1")[4].text
  except:
    color = "missing"

  try:
    mileage = soup.find_all("td", class_="css-7whdrf ezjvm5n1")[5].text.replace("\xa0", "")
  except:
    mileage = "missing"

  try:
    steering_wheel = soup.find_all("td", class_="css-7whdrf ezjvm5n1")[6].text
  except:
    steering_wheel = "missing"

  try:
    generation_style = soup.find_all("a", class_="css-1n9bvfr e1oy5ngb0")[0].text
  except:
    generation_style = "missing"

  try:
    vehicle_configuration = soup.find_all("a", class_="css-1n9bvfr e1oy5ngb0")[1].text
  except:
    vehicle_configuration = "missing"

  try:
    vechicle_logbook_check = soup.find_all("a", class_="css-17f5zdi e1wvjnck0")[4].text
  except:
    vechicle_logbook_check = "missing"

  try:
    number_of_records_in_vechicle_logbook = soup.find_all("a", class_="css-17f5zdi e1wvjnck0")[5].text[:2].replace(" ", "")
  except:
    number_of_records_in_vechicle_logbook = "missing"

  try:
    car_wanted_check = soup.find_all("div", class_="css-13qo6o5 ev29ov71")[2].text
  except:
    car_wanted_check = "missing"

  try:
    legal_restrictions = soup.find_all("div", class_="css-13qo6o5 ev29ov71")[3].text
  except:
    legal_restrictions = "missing"

  try:
    publication_date = soup.find("div", class_="css-pxeubi evnwjo70").text[-10:]
  except:
    publication_date = "missing"

  try:
    number_of_views = soup.find_all("div", class_="css-14wh0pm e1lm3vns0")[0].text
  except:
    number_of_views = "missing"

  car_data.append([car_link, brand, model, manufacture_year, sellers_city, price, fuel_type, engine_size, hp_power, gearbox_type,
                   transmission_type, color, mileage, steering_wheel, generation_style, vehicle_configuration,
                   vechicle_logbook_check, number_of_records_in_vechicle_logbook, car_wanted_check, legal_restrictions,
                   publication_date, number_of_views])

100%|██████████| 18892/18892 [2:40:06<00:00,  1.97it/s]  


In [ ]:
df_cars = pd.DataFrame(car_data, columns=["link", "brand", "model", "year_of_manufacture", "location", "price", "fuel_type", "engine_size", "hp_power", "gearbox",
                                       "transmission", "color", "mileage", "steering_wheel", "generation_style", "vehicle_config", "vechicle_logbook_check",
                                       "cnt_of_records_in_vechicle_logbook", "car_wanted_check", "legal_restrictions", "publication_date", "number_of_views"])

In [ ]:
df_cars.to_csv('cars_dom.csv')